# Цели Исследования

Нашей компанией приобретена база данных крупного сервиса для чтения книг. Их необходимо проанализировать и ответить на следующие вопросы:
- Сколько книг вышло после 1 января 2000 года?
- Для каждой книги вывести количество обзоров и среднюю оценку
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц
- Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками
- Какое среднее количество обзоров от пользователей, которые поставили больше 50 оценок?

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine


In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Посчитайте, сколько книг вышло после 1 января 2000 года;

In [4]:
df1 = pd.io.sql.read_sql(""" 
SELECT COUNT(book_id) 
FROM books 
WHERE EXTRACT (YEAR FROM CAST (publication_date AS date)) >= 2000
""" , con = engine)
df1

,count
0,821


821 книга

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [5]:
df2 = pd.io.sql.read_sql(""" 
WITH a AS(SELECT b.title, b.book_id, 
       COUNT(rev.review_id)
FROM books AS b
INNER JOIN reviews AS rev ON b.book_id = rev.book_id
GROUP BY b.title, b.book_id)

SELECT a.title,
       a.count,
       AVG(rat.rating)
FROM a
INNER JOIN ratings AS rat ON a.book_id = rat.book_id
GROUP BY a.title, a.count



""" , con = engine)
df2

,title,count,avg
0,V for Vendetta,3,4.222222
1,The Fiery Cross (Outlander #5),4,4.200000
2,The Canterbury Tales,3,3.333333
3,Burning Chrome (Sprawl #0),2,4.500000
4,The Summons,3,4.333333
...,...,...,...
989,The Notebook (The Notebook #1),5,3.868421
990,The Innocent Man: Murder and Injustice in a Sm...,2,4.500000
991,Rules of Prey (Lucas Davenport #1),3,3.333333
992,A Hunger Like No Other (Immortals After Dark #1),2,4.000000


Посчитано

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [7]:
df3 = pd.io.sql.read_sql(""" 
SELECT p.publisher,
       COUNT(b.title)
FROM books AS b
INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY COUNT(b.title) DESC
LIMIT 1




""" , con = engine)
df3

,publisher,count
0,Penguin Books,42


Это издательство: Penguin Books

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:
df4 = pd.io.sql.read_sql(""" 
WITH c AS (SELECT b.book_id, a.author, AVG(rat.rating) AS avg_rating, COUNT(rat.rating_id)
FROM books AS b
JOIN ratings AS rat ON b.book_id = rat.book_id
JOIN authors AS a ON a.author_id = b.author_id
GROUP BY b.book_id, a.author
HAVING COUNT(rat.rating_id) >= 50)

SELECT c.author, 
       AVG(avg_rating),
       COUNT(c.count)
       
FROM c
GROUP BY c.author
LIMIT 1

""" , con = engine)
df4

,author,avg,count
0,J.K. Rowling/Mary GrandPré,4.283844,4


Автор с самой высокой средней оценкой J.K. Rowling/Mary GrandPré 4.283844

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
df5 = pd.io.sql.read_sql("""
WITH c AS (WITH a AS (SELECT username, 
COUNT(rating_id)
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 50)

SELECT COUNT(rev.review_id)
FROM a
INNER JOIN reviews AS rev ON a.username = rev.username
GROUP BY a.username)

SELECT AVG(count)
FROM c

""" , con = engine)
df5

,avg
0,24.333333


Среднее количество обзоров от пользователей которые поставили больше 50 оценок: 24,3

# Общий вывод

База данных исследована, ответы на вопросы получены. Можно смело создавать новый продукт :)